In [2]:
import h5py
import numpy as np
import pandas as pd
import scipy as sp
import networkx as nx
if nx.__version__ == '2.8.8':
    import warnings 
    warnings.simplefilter(action='ignore', category=FutureWarning)
from node_classification import node_categories
names_node_cats = ['bulk', 'root', 'dense sprout', 'sparse sprout', 'inner tree node', 'proper leaf']
import copy

In [3]:

def compute_WS_grid_measures(G, A, P, nn, diffs, flows):
    
    loads = np.abs(flows)
    max_load_idx = np.argmax(loads)
    load_idx = list(list(G.edges)[max_load_idx])
    
    # For backup measures, i.e., post failure of maximally loaded line
    G_post = copy.deepcopy(G)
    G_post.remove_edge(*load_idx)
    post_connected = nx.is_connected(G_post)

    df = pd.DataFrame()
    
    # node features
    
    df["node_number"] = nn

    df["P"] = P
    
    df["AP"] = A.dot(P)
    
    df["AAP"] = A.dot(df["AP"])
    
    df["AAAP"] = A.dot(df["AAP"])
    
    df["row_sum_AA"] = A.dot(A.dot(np.ones(len(P))))
    
    df["row_sum_AAA"] = A.dot(df["row_sum_AA"])
    
    df["degree"] = [d[1] for d in G.degree]
    
    df["degree_centrality"] = nx.degree_centrality(G).values()
    
    df["neighbor_degree_min"] = [min(df["degree"][G.neighbors(node)]) for node in G.nodes]
    
    df["neighbor_degree_max"] = [max(df["degree"][G.neighbors(node)]) for node in G.nodes]
    
    df["neighbor_degree_mean"] = [np.array(df["degree"][G.neighbors(node)]).mean() for node in G.nodes]
    
    df["clustering"] = nx.clustering(G).values()
    
    df["betweenness_centrality"] = nx.betweenness_centrality(G).values()
    
    df["closeness_centrality"] = nx.closeness_centrality(G).values()
    
    df["load_centrality"] = nx.load_centrality(G).values()
    
    df["eigenvector_centrality"] = nx.eigenvector_centrality(G,max_iter=10000).values()
    
    df["katz_centrality"] = nx.katz_centrality(G).values()
    
    df["second_order_centrality"] = nx.second_order_centrality(G).values()
    
    df["current_flow_closeness_centrality"] = nx.current_flow_closeness_centrality(G).values()
    
    df["current_flow_betweenness_centrality"] = nx.current_flow_betweenness_centrality(G).values()
    
    df["average_neighbor_degree"] = nx.average_neighbor_degree(G).values()
    
    df["harmonic_centrality"] = nx.harmonic_centrality(G).values()
    
    df["square_clustering"] = nx.square_clustering(G).values()
    
    df["eccentricity"] = nx.eccentricity(G).values()
    
    # This redundant feature is only here for backwards compatibility
    df["node_cat"] = node_categories(G, denseThres=5).values()
    for cat in names_node_cats:
#         ddf[cat] = [node == cat for node in df["node_cat"]] -> df[cat] = [int(node == cat) for node in df["node_cat"]
        df[cat] = [int(node == cat) for node in df["node_cat"]]

        
    # new node features
    
    df["fiedler_vector"] = nx.fiedler_vector(G)
    
    connected_to_max_load_line = np.zeros(len(G), dtype=bool)
    connected_to_max_load_line[load_idx] = True
    df["node_connected_to_max_load_line"] = connected_to_max_load_line
    
    # Assign an index to every edge, 
    # find indices of connected edges for very node (e[2]), 
    # use indices to look up corresponding loads
    nx.set_edge_attributes(G, dict(zip(G.edges, range(len(G.edges())))), "edge_idx")
    df["max_load_connected_lines"] = [loads[[e[2] for e in G.edges(nbunch=node, data="edge_idx")]].max() / 9.0 for node in G.nodes]
    df["min_load_connected_lines"] = [loads[[e[2] for e in G.edges(nbunch=node, data="edge_idx")]].min() / 9.0 for node in G.nodes] 
    df["mean_load_connected_lines"] = [loads[[e[2] for e in G.edges(nbunch=node, data="edge_idx")]].mean() / 9.0 for node in G.nodes] 
    
    # nx.set_edge_attributes(G, dict(zip(G.edges, 9 * np.cos(diffs))), "laplace_weights")
    df["resistance_distance_centrality"] = nx.current_flow_closeness_centrality(G, weight="laplace_weights").values()
    
    # graph features
    
    N = len(G.nodes)
    M = len(G.edges)
    
    df["degree_assortativity_coefficient"]= nx.degree_assortativity_coefficient(G)
    
    df["transitivity"] = nx.transitivity(G)
    
    df["diameter"] = nx.diameter(G)
    
    nx.set_node_attributes(G, dict(zip(G.nodes, P)), "P")
    
    df["attribute_assortativity_coefficient_P"] = nx.attribute_assortativity_coefficient(G, "P")
    
    df["kirchhoff_index"] = np.reciprocal(df["current_flow_closeness_centrality"]).mean()
    
    df["resistance_distance_kirchhoff_index"] = np.reciprocal(df["resistance_distance_centrality"]).mean()
    
    # new graph features
    
    spec = nx.laplacian_spectrum(G)
    
    df["inverse_algebraic_connectivity"] = 1 / spec[1]
    
    df["eigenratio"] = spec[1] / spec[-1]
     
    df["power_sign_ratio"] = (sum([-P[e[0]] * P[e[1]] for e in G.edges]) + M) / (2 * M)
    
    df["maximal_line_load"] = loads[max_load_idx] / 9.0
    
    df["universal_kuramoto_order_parameter"] = np.cos(diffs).sum() / M
    
    df["connected_post"] = post_connected
    if post_connected:
        # Post failure steady state in DC approximation, i.e, P = L theta
        theta_post = sp.sparse.linalg.lsqr(nx.laplacian_matrix(G_post),P)[0]
        theta_post = theta_post - theta_post[0] # choose first theta as 0
        diffs_post = [theta_post[e[0]] - theta_post[e[1]] for e in G_post.edges]
        loads_post = np.abs(9 * np.sin(diffs_post))      
        # Line outage distribution factor (LODF) in DC approximation
        df["maximal_line_load_post_dc"] = loads_post.max() / 9.0
        df["backup_capacity"] = (loads_post - np.delete(loads, max_load_idx)).max()       
    else:
        df["maximal_line_load_post_dc"] = np.nan
        df["backup_capacity"] = np.nan
  
    return df


In [46]:
f = h5py.File("n128/data/ml_input_grid_data_homo.h5", "r")
_P = open("n128/data/grids/state_00001.json", "r")
P=np.array(eval(_P.read()))
_P.close()
P

array([ 9.99550437e-01, -2.99820488e-02,  9.99894798e-01, -1.45049355e-02,
        0.00000000e+00,  9.99850424e-01, -1.72953598e-02,  9.97541507e-01,
       -7.00781073e-02,  0.00000000e+00,  9.99973736e-01, -7.24755111e-03,
        9.97468264e-01, -7.11130265e-02,  9.99903390e-01, -1.38999965e-02,
        0.00000000e+00,  9.98572714e-01, -5.34091280e-02,  0.00000000e+00,
        9.98034751e-01, -6.26628806e-02,  0.00000000e+00,  9.96557845e-01,
       -8.29003079e-02,  0.00000000e+00,  9.99931731e-01, -1.16847413e-02,
        9.99904919e-01, -1.37895995e-02,  0.00000000e+00,  9.98764230e-01,
       -4.96992214e-02,  9.98497216e-01, -5.48024657e-02,  0.00000000e+00,
        9.98523867e-01, -5.43147093e-02,  0.00000000e+00,  9.98586788e-01,
       -5.31453444e-02,  9.98034961e-01, -6.26595288e-02,  0.00000000e+00,
        9.98172672e-01, -6.04261267e-02,  9.99893615e-01, -1.45862353e-02,
        0.00000000e+00,  9.99618992e-01, -2.76020224e-02,  9.96467093e-01,
       -8.39841254e-02,  

SyntaxError: unexpected EOF while parsing (<string>, line 0)

In [14]:
edges = np.array(f["grids/1/edge_index"])
n_nodes = 128

G = nx.empty_graph(n_nodes)
for i, j in edges:
    G.add_edge(i - 1, j - 1) # -1 from Julia to Python

A = nx.adjacency_matrix(G)


186

Docstring:
fromfile(file, dtype=float, count=-1, sep='', offset=0, *, like=None)

Construct an array from data in a text or binary file.

A highly efficient way of reading binary data with a known data-type,
as well as parsing simply formatted text files.  Data written using the
`tofile` method can be read using this function.

Parameters
----------
file : file or str or Path
    Open file object or filename.

    .. versionchanged:: 1.17.0
        `pathlib.Path` objects are now accepted.

dtype : data-type
    Data type of the returned array.
    For binary files, it is used to determine the size and byte-order
    of the items in the file.
    Most builtin numeric types are supported and extension types may be supported.

    .. versionadded:: 1.18.0
        Complex dtypes.

count : int
    Number of items to read. ``-1`` means all items (i.e., the complete
    file).
sep : str
    Separator between items if file is a text file.
    Empty ("") separator means the file should be treat